In [ ]:
import copy
import collections
import string
from pandas import *
from string import digits
import re
import math
import csv
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
n_gram = 4  #ng-ramm modell festlegen

textXXL = ""

In [ ]:
# bitte die Pfade ändern! hier und für die Tests ganz unten.
# Bei Fehlermeldung bitte <encoding="utf8"> im open Kopf einfügen // ich nutze MacOS
#x_train.txt
with open('/Users/abdullah/Documents/WS2223/Sprach:Text verstehen/A2_Umsetzung/wili-2018/x_train.txt','r') as f:
   texte = f.readlines()
f.close()

In [ ]:
#y_train.txt
with open('/Users/abdullah/Documents/WS2223/Sprach:Text verstehen/A2_Umsetzung/wili-2018/y_train.txt','r') as f:
    sprachen_id = f.readlines()
f.close()

In [ ]:
#labels.csv
sprachen_ausgeschrieben = {}
data_new = []

with open('/Users/abdullah/Documents/WS2223/Sprach:Text verstehen/A2_Umsetzung/wili-2018/labels.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
for i in data:
    i = i[0].split(";")
    data_new.append(i)

for i in range (1,len(data_new)):
    sprachen_ausgeschrieben[data_new[i][0]] = data_new[i][4]


##### Text vorverarbeiten 

In [ ]:
for i in range(len(sprachen_id)):
    sprachen_id[i] = sprachen_id[i].replace("\n","")

In [ ]:
dict = {}
for i in range(len(texte)):
    if sprachen_id[i] in dict :
        old = dict[sprachen_id[i]]
        dict[sprachen_id[i]] = old + " " + texte[i]
    else:
        dict[sprachen_id[i]] = texte[i]

In [ ]:
def text_vorverarbeiten(text):
    text = re.sub(r'[/\\!;:<>@^#€$()۔,’„.–“"=&§%^*?+_\':;~`0-9]', '', text)
    text = text.replace("،", " ")
    text = text.replace("-", " ")
    text = text.replace("—", " ")
    text = text.replace('\u202c',' ')
    text = text.replace('\u200c',' ')
    text = re.sub(r'\s+', ' ',   text)  
    text = text.lower()

    
    return text

In [ ]:
for sprache in dict.keys():
    dict[sprache] = text_vorverarbeiten(dict[sprache])
    textXXL += " " + dict[sprache]
textXXL = text_vorverarbeiten(textXXL)

In [ ]:
def ngrams( content, n):
    d = {}
    length = len( content )
    for i in range( 0, length - n + 1 ):
        k = content[i:i + n]
        if k not in d:
            d[k] = 0
        d[k] += 1
    return d

In [ ]:
def ngram_vocab( content, n):
    d = {}
    length = len( content )
    for i in range( 0, length - n + 1 ):
        k = content[i:i + n]
        if k not in d:
            d[k] = ''
    return d

In [ ]:
def ngram_ohne_count( content, n):
    d = []
    length = len( content )
    for i in range( 0, length - n + 1 ):
        k = content[i:i + n]
        d.append(k)

    return d

In [ ]:
vocab = ngram_vocab(textXXL,n_gram)
sprachen_ngrams = []
texte = list(dict.items())
for l in range (len(dict)): # l = language
    text = texte[l][1] # je sprache
    n_grams = ngrams(text,n_gram)
    sprachen_ngrams.append((texte[l][0],n_grams))

In [ ]:
len_vocab = len(vocab)
print(len_vocab)
len_vocan_sprachen = {}
for i in range(len(sprachen_ngrams)) :
   len_vocan_sprachen[sprachen_ngrams[i][0]] = sum(sprachen_ngrams[i][1].values())


##### Wahrscheinlichkeiten berechnen 

In [ ]:
#Naive Bayes-Klassifikator & Laplace(add-1)smoothing
dic_wahrscheinlichkeiten = []
for i in range(len(sprachen_ngrams)):
    sprache_dic = {}
    for g in sprachen_ngrams[i][1].keys():
        sprache_dic[g] = ((1+sprachen_ngrams[i][1][g])/(len_vocab + len_vocan_sprachen[sprachen_ngrams[i][0]]))
    dic_wahrscheinlichkeiten.append((sprachen_ngrams[i][0], sprache_dic))

In [ ]:
#log-wahrscheinlichkeit
def Wahrscheinlichkeit_sprache(bigrams, sprache_wahrscheinlichkeiten,name_der_Sprache):
    wk = 0.0
    for test_text in bigrams:  
        if test_text in sprache_wahrscheinlichkeiten.keys():
            wk += math.log(sprache_wahrscheinlichkeiten[test_text],2)
        else: 
            wk += math.log((1/(len_vocab + len_vocan_sprachen[name_der_Sprache])),2)
            #wk += math.log(10e-25,2)
    return wk*(1/len(dict))

In [ ]:
def wahrscheinlichkeiten_berechnen(text):
    wahrscheinlichkeiten = []
    dic_test_text = ngram_ohne_count(text,n_gram)
    for i in range (len(dic_wahrscheinlichkeiten)): #i ist eine sprache
       wahrscheinlichkeiten.append((dic_wahrscheinlichkeiten[i][0], Wahrscheinlichkeit_sprache(dic_test_text,dic_wahrscheinlichkeiten[i][1],dic_wahrscheinlichkeiten[i][0])))

    return wahrscheinlichkeiten

In [ ]:
def get_language_name(d):
    new_d = {}
    for i in d: 
        new_key = sprachen_ausgeschrieben[i]
        new_d[new_key] = d[i]       
    return new_d

In [ ]:
def schaetze(text,sprachennameausgeschrieben=False):
   # text = text_vorverarbeiten(text) -----> # die Tests die ich ausgeführt habe habe ich bereits schon vorbereitet deswegen ist die Zeile hier auskommentiert ansonsten bevor fremde tests durchgeführt werden -> text vorverarbeiten!
    res1 =  wahrscheinlichkeiten_berechnen(text)
    res1.sort(key=lambda x:x[1],reverse = True)
    res1 = res1[:3]

    dic_res = {}
    dic_res[res1[0][0]] = res1[0][1]
    dic_res[res1[1][0]] = res1[1][1]
    dic_res[res1[2][0]] = res1[2][1]
    if sprachennameausgeschrieben:
      dic_res= get_language_name(dic_res)
    return dic_res

#### einen schnellen Test ausführen, text1 = irgendeine gewollte Sprache zum schätzen 

In [30]:
text1=  "shqptar"
text1 = text_vorverarbeiten(text1)
res1 = schaetze(text1,True)

print(res1)

{'Albanisch': -0.3388599231463731, 'Nahuatl': -0.3519302410504222, 'Interlingua': -0.35348102973938933}


### Testen:


In [ ]:
#x_test.txt
with open('/Users/abdullah/Documents/WS2223/Sprach:Text verstehen/A2_Umsetzung/wili-2018/x_test.txt','r') as f:
   test_texte = f.readlines()
f.close()


In [ ]:
#y_test.txt
with open('/Users/abdullah/Documents/WS2223/Sprach:Text verstehen/A2_Umsetzung/wili-2018/y_test.txt','r') as f:
    test_sprachen_id = f.readlines()
f.close()

In [ ]:
for i in range(len(test_sprachen_id)):
    test_sprachen_id[i] = test_sprachen_id[i].replace("\n","")

In [ ]:
for i in range(len(test_texte)):
    test_texte[i] =  text_vorverarbeiten(test_texte[i])


In [ ]:
test_ergbnisse= {}
for i in range(len(test_sprachen_id)):
    if (test_sprachen_id[i] not in test_ergbnisse):
        test_ergbnisse[test_sprachen_id[i]] = 0
    else:
        continue

##### Achtung dieser Test läuft ca. 44 min -> weil der Testdatensatz erheblich groß ist (500*235)

In [ ]:
for i in range(len(test_sprachen_id)):
    ergebnis = schaetze(test_texte[i])
    if test_sprachen_id[i] in ergebnis:
        test_ergbnisse[test_sprachen_id[i]] += 1

In [ ]:
ergebnis_xy = {}
for i in test_ergbnisse.keys():
    ergebnis_xy[i]=(test_ergbnisse[i], test_sprachen_id.count(i),((test_ergbnisse[i]/test_sprachen_id.count(i))*100.0))

In [ ]:
# Die durchschnittliche Genauigkeit 
sum_3 = 0
for i in ergebnis_xy:
    sum_3 += ergebnis_xy[i][2]

mtlwert = sum_3/len(ergebnis_xy)    

print(mtlwert)

In [ ]:
print(ergebnis_xy) # (erfolgreiche Tests,Anzahl tests insgesamt, %)

#### für den Fall, dass Sie den Code laufen lassen, hier sind meine Ergbnisse zum Vergleich! 
{'mwl': (495, 500, 99.0), 'nld': (492, 500, 98.4), 'ava': (367, 500, 73.4), 'tcy': (500, 500, 100.0), 'bjn': (475, 500, 95.0), 'mon': (498, 500, 99.6), 'glk': (482, 500, 96.39999999999999), 'lez': (484, 500, 96.8), 'bul': (482, 500, 96.39999999999999), 'nan': (492, 500, 98.4), 'sco': (496, 500, 99.2), 'lij': (495, 500, 99.0), 'ido': (490, 500, 98.0), 'ara': (497, 500, 99.4), 'kan': (492, 500, 98.4), 'hat': (491, 500, 98.2), 'bar': (460, 500, 92.0), 'mri': (500, 500, 100.0), 'kok': (461, 500, 92.2), 'kor': (489, 500, 97.8), 'koi': (466, 500, 93.2), 'mya': (498, 500, 99.6), 'ind': (490, 500, 98.0), 'jpn': (473, 500, 94.6), 'kin': (477, 500, 95.39999999999999), 'be-tarask': (494, 500, 98.8), 'egl': (486, 500, 97.2), 'mzn': (489, 500, 97.8), 'eus': (497, 500, 99.4), 'cat': (479, 500, 95.8), 'nds-nl': (498, 500, 99.6), 'tsn': (491, 500, 98.2), 'sin': (470, 500, 94.0), 'pnb': (495, 500, 99.0), 'fin': (497, 500, 99.4), 'min': (498, 500, 99.6), 'stq': (490, 500, 98.0), 'zho': (387, 500, 77.4), 'rus': (499, 500, 99.8), 'kur': (494, 500, 98.8), 'chv': (485, 500, 97.0), 'orm': (485, 500, 97.0), 'ell': (494, 500, 98.8), 'nob': (496, 500, 99.2), 'spa': (489, 500, 97.8), 'pag': (441, 500, 88.2), 'rup': (444, 500, 88.8), 'hye': (489, 500, 97.8), 'fra': (497, 500, 99.4), 'csb': (497, 500, 99.4), 'san': (495, 500, 99.0), 'lzh': (429, 500, 85.8), 'cym': (497, 500, 99.4), 'als': (495, 500, 99.0), 'snd': (496, 500, 99.2), 'nrm': (495, 500, 99.0), 'kir': (494, 500, 98.8), 'hak': (484, 500, 96.8), 'vol': (485, 500, 97.0), 'zh-yue': (323, 500, 64.60000000000001), 'eng': (499, 500, 99.8), 'gle': (490, 500, 98.0), 'vls': (491, 500, 98.2), 'yid': (491, 500, 98.2), 'mal': (492, 500, 98.4), 'asm': (485, 500, 97.0), 'srp': (455, 500, 91.0), 'dsb': (493, 500, 98.6), 'slv': (485, 500, 97.0), 'yor': (440, 500, 88.0), 'jav': (480, 500, 96.0), 'lao': (479, 500, 95.8), 'vro': (489, 500, 97.8), 'guj': (482, 500, 96.39999999999999), 'bre': (493, 500, 98.6), 'uzb': (497, 500, 99.4), 'rue': (494, 500, 98.8), 'chr': (492, 500, 98.4), 'ben': (441, 500, 88.2), 'mrj': (489, 500, 97.8), 'aze': (489, 500, 97.8), 'lim': (497, 500, 99.4), 'frp': (488, 500, 97.6), 'krc': (487, 500, 97.39999999999999), 'mlg': (500, 500, 100.0), 'msa': (489, 500, 97.8), 'ksh': (484, 500, 96.8), 'hin': (488, 500, 97.6), 'khm': (441, 500, 88.2), 'glv': (491, 500, 98.2), 'lug': (490, 500, 98.0), 'que': (475, 500, 95.0), 'zea': (495, 500, 99.0), 'arz': (497, 500, 99.4), 'arg': (497, 500, 99.4), 'kab': (466, 500, 93.2), 'wuu': (370, 500, 74.0), 'bod': (499, 500, 99.8), 'ita': (495, 500, 99.0), 'wol': (468, 500, 93.60000000000001), 'fry': (491, 500, 98.2), 'ast': (493, 500, 98.6), 'afr': (498, 500, 99.6), 'srd': (487, 500, 97.39999999999999), 'sqi': (489, 500, 97.8), 'oci': (496, 500, 99.2), 'gag': (495, 500, 99.0), 'cor': (498, 500, 99.6), 'slk': (499, 500, 99.8), 'swa': (481, 500, 96.2), 'lin': (466, 500, 93.2), 'pdc': (478, 500, 95.6), 'grn': (483, 500, 96.6), 'mai': (488, 500, 97.6), 'hau': (496, 500, 99.2), 'nci': (468, 500, 93.60000000000001), 'war': (498, 500, 99.6), 'sme': (492, 500, 98.4), 'ang': (455, 500, 91.0), 'cos': (495, 500, 99.0), 'mar': (478, 500, 95.6), 'bpy': (497, 500, 99.4), 'che': (496, 500, 99.2), 'sna': (475, 500, 95.0), 'scn': (498, 500, 99.6), 'xmf': (486, 500, 97.2), 'tam': (494, 500, 98.8), 'urd': (493, 500, 98.6), 'nds': (496, 500, 99.2), 'diq': (493, 500, 98.6), 'tel': (471, 500, 94.19999999999999), 'vep': (499, 500, 99.8), 'hrv': (496, 500, 99.2), 'ltg': (497, 500, 99.4), 'ibo': (440, 500, 88.0), 'gla': (490, 500, 98.0), 'vie': (491, 500, 98.2), 'bho': (480, 500, 96.0), 'pcd': (404, 500, 80.80000000000001), 'lrc': (488, 500, 97.6), 'azb': (499, 500, 99.8), 'lav': (489, 500, 97.8), 'oss': (499, 500, 99.8), 'mdf': (483, 500, 96.6), 'heb': (499, 500, 99.8), 'amh': (494, 500, 98.8), 'bel': (486, 500, 97.2), 'kat': (491, 500, 98.2), 'kaa': (498, 500, 99.6), 'tat': (483, 500, 96.6), 'nno': (497, 500, 99.4), 'new': (487, 500, 97.39999999999999), 'cbk': (497, 500, 99.4), 'sun': (474, 500, 94.8), 'ori': (479, 500, 95.8), 'tuk': (498, 500, 99.6), 'xho': (473, 500, 94.6), 'div': (499, 500, 99.8), 'ina': (495, 500, 99.0), 'jam': (493, 500, 98.6), 'hsb': (486, 500, 97.2), 'uig': (493, 500, 98.6), 'fur': (486, 500, 97.2), 'crh': (495, 500, 99.0), 'ile': (460, 500, 92.0), 'vec': (495, 500, 99.0), 'lad': (492, 500, 98.4), 'pap': (487, 500, 97.39999999999999), 'mlt': (497, 500, 99.4), 'mkd': (488, 500, 97.6), 'por': (480, 500, 96.0), 'ltz': (474, 500, 94.8), 'roa-tara': (483, 500, 96.6), 'epo': (490, 500, 98.0), 'srn': (486, 500, 97.2), 'myv': (486, 500, 97.2), 'bak': (491, 500, 98.2), 'swe': (498, 500, 99.6), 'nap': (494, 500, 98.8), 'tgl': (493, 500, 98.6), 'glg': (492, 500, 98.4), 'map-bms': (497, 500, 99.4), 'dan': (490, 500, 98.0), 'ces': (493, 500, 98.6), 'bcl': (484, 500, 96.8), 'deu': (491, 500, 98.2), 'est': (484, 500, 96.8), 'pan': (492, 500, 98.4), 'isl': (499, 500, 99.8), 'fao': (498, 500, 99.6), 'bxr': (469, 500, 93.8), 'pam': (479, 500, 95.8), 'tyv': (491, 500, 98.2), 'ukr': (492, 500, 98.4), 'aym': (466, 500, 93.2), 'pol': (498, 500, 99.6), 'dty': (497, 500, 99.4), 'som': (480, 500, 96.0), 'fas': (499, 500, 99.8), 'wln': (498, 500, 99.6), 'sgs': (500, 500, 100.0), 'bos': (496, 500, 99.2), 'szl': (496, 500, 99.2), 'pfl': (490, 500, 98.0), 'hbs': (481, 500, 96.2), 'pus': (480, 500, 96.0), 'kaz': (499, 500, 99.8), 'jbo': (500, 500, 100.0), 'kom': (470, 500, 94.0), 'ron': (494, 500, 98.8), 'nep': (491, 500, 98.2), 'hif': (464, 500, 92.80000000000001), 'sah': (494, 500, 98.8), 'ilo': (483, 500, 96.6), 'hun': (494, 500, 98.8), 'nav': (500, 500, 100.0), 'kbd': (500, 500, 100.0), 'tha': (494, 500, 98.8), 'ext': (479, 500, 95.8), 'tet': (448, 500, 89.60000000000001), 'ace': (494, 500, 98.8), 'tur': (495, 500, 99.0), 'cdo': (480, 500, 96.0), 'mhr': (487, 500, 97.39999999999999), 'tgk': (488, 500, 97.6), 'lat': (470, 500, 94.0), 'lit': (492, 500, 98.4), 'ceb': (500, 500, 100.0), 'lmo': (456, 500, 91.2), 'olo': (496, 500, 99.2), 'ton': (497, 500, 99.4), 'nso': (485, 500, 97.0), 'roh': (489, 500, 97.8), 'udm': (490, 500, 98.0), 'ckb': (498, 500, 99.6)}

## Visualisierung 

In [ ]:

wichtige_Sprache = {}
wichtige_Sprache["spa"] = 0
wichtige_Sprache["ara"] = 0
wichtige_Sprache["deu"] = 0
wichtige_Sprache["eng"] = 0
wichtige_Sprache["tcy"] = 0
wichtige_Sprache["hin"] = 0
wichtige_Sprache["jpn"] = 0
wichtige_Sprache["ita"] = 0
wichtige_Sprache["lat"] = 0
wichtige_Sprache["rus"] = 0
wichtige_Sprache["zho"] = 0
wichtige_Sprache["fra"] = 0
wichtige_Sprache["por"] = 0


for i in ergebnis_xy.keys():
    if i in wichtige_Sprache:
        wichtige_Sprache[i]= ergebnis_xy[i]

print(wichtige_Sprache)

In [ ]:
l = []
for i in wichtige_Sprache:
    l.append(wichtige_Sprache[i][2])

ywerte = l
xwerte = get_language_name(wichtige_Sprache).keys()

fig = plt.figure(figsize = (20, 10)) 
g = 20
plt.rc ('font', size = g) # steuert die Standardtextgröße
plt.rc ('axes', titlesize = g) # Schriftgröße des Titels
plt.rc ('axes', labelsize = g) # Schriftgröße der x- und y-Beschriftungen
plt.rc ('xtick', labelsize = g) #Schriftgröße der x-Tick-Labels
plt.rc ('ytick', labelsize = g-5) #Schriftgröße der y-Tick-Labels
plt.rc ('legend', fontsize = g) 
# plt.axhline()
#plt.bar(xwerte, ywerte,color = "Blue",  width = 0.5)
plt.xticks(rotation=45)
plt.yticks(np.arange(0, 100.5, 1))
plt.xlabel("Sprache")
plt.ylabel("Der Anteil der erfolgreichen Tests in %")

plt.rc('grid', linestyle=':', color='black', linewidth=2)
plt.scatter(xwerte, ywerte,color="blue",s=80)
plt.grid(True)
plt.show()